# Pitch Identification

In [ ]:
#!pip install ffmpeg

In [ ]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

In [ ]:
filename = 'audio/training/J_cata_02.wav'
fs, audio1 = read(filename)
f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')
print('filename: ', filename)
print('Data Length (s): ',t[-1])
print('Sampling frequency (samples/s): ', fs)

#Playing Audio (Reproduce Audio)
ipd.Audio(filename)

In [ ]:
#Spectogram
pyplot.rcParams['figure.figsize'] = 14,5
pyplot.pcolormesh(t, f[:450], S1[:450][:])
pyplot.title("Spectrogram")
xlabel('time(s)')
ylabel('frequency(Hz)')
#show()

In [ ]:
#Filtro ButterWord

wn = 1500/(fs/2)
b3,a3 = butter(4, wn)
audio1_filt = lfilter(b3,a3,audio1)

In [ ]:
pyplot.rcParams['figure.figsize'] = 16,5
#plot(audio1,'r')
plot(absolute(audio1_filt),'b')

In [ ]:
#result = [1 if item > 0.0004*1e8 else 0 for item in audio1_filt**2]
#plot(result)

# Fix-sized segmentation





In the first part we showed how we can segment a long recording to non-overlapping segments using ffmpeg. The following code sample shows how to do the same with Python. Line 8 does the actual segmentation in a single-line command. Overall, the following script loads and normalizes an audio signal, and then it breaks it into 1-second segments and writes each one of them in a file.





(Pay attention to the note in the last comment: you will need to cast to 16bit before saving to file because the numpy conversion has led to higher sample resolutions).

In [ ]:

# Fix-sized segmentation (breaks a signal into non-overlapping segments)
    #signal = audio1 / (2**15)
    #signal_len = len(signal)
    #segment_size_t = 1 # segment size in seconds
    #segment_size = segment_size_t * fs  # segment size in samples

# Break signal into list of segments in a single-line Python code
    #segments = np.array([signal[x:x + segment_size] for x in
    #                     np.arange(0, signal_len, segment_size)])

# Save each segment in a seperate filename
    #for iS, s in enumerate(segments):
    #    wavfile.write("data/obama_segment_{0:d}_{1:d}.wav".format(segment_size_t * iS,
    #                                                              segment_size_t * (iS + 1)), fs, (s))

# Audio Division 

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

song = AudioSegment.from_wav(filename)

db = -30
segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
while len(segments) != 2:
    db = db-1
    segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)

    
n = len(segments)
print("Size of List", n)


# Process each chunk with your parameters
for i, segment in enumerate(segments):

    # Export the audio chunk with new bitrate.
    print("Exporting chunk{0}.wav.".format(i))
    segment.export(
        "audio/data/chunk{0}.wav".format(i),
        bitrate = "192k",
        format = "wav"
    )


# Pitch Detection 

    Using  the cepstrum coeficient. 

In [ ]:
import numpy as np

#Cepstrum and Power Spectrum

def real_cepstrum(x, n=None):
# Compute the real cepstrum of a real sequence
#    x : ndarray
#        Real sequence to compute real cepstrum of.
#    n : {None, int}, optional
#        Length of the Fourier transform.
#    Returns
#    -------
#    ceps: ndarray
#        The real cepstrum.

    spectrum = np.abs(np.fft.fft(x, n=n))**2
    ceps = np.fft.ifft(np.log(spectrum))

    return ceps, spectrum

In [ ]:
#Zero Crossing Rate

def ZCR(samples, frameSize, overlap):
    wlen = len(samples)
    step = frameSize - overlap
    frameNum = math.ceil(wlen/step)
    zcr = np.zeros((frameNum,1))
    for i in range(frameNum):
        curFrame = samples[np.arange(i*step,min(i*step+frameSize,wlen))]
        #To avoid DC bias, usually we need to perform mean substraction on each frame
        curFrame = curFrame - np.mean(curFrame) #Zero-Justified
        zcr[i] = sum(curFrame[0:-1]*curFrame[1::]<=0)
    return zcr

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

samples = segments[0].get_array_of_samples()
samples = np.array(samples)

#id_filename = 'audio/data/chunk0.wav'
#fs, id_audio = read(id_filename)

#Calculate Cepstrung and Power Spectrum
ceps, spec = real_cepstrum(samples, n=None)
N = samples.shape[0]

#Calculate Zero Cross Rate
frameSize = 256
overlap = 0
zcr = ZCR(samples, frameSize, overlap)

In [ ]:
#Graphic the samples 
plt.rcParams['figure.figsize'] = 16,5
plot(zcr,'b')

In [ ]:
a = 0
for x in zcr:
    a+= x
    
b = round(a[0])

print(round(np.average(zcr)))

print(b)

In [ ]:
#Power Specturm
power_spec = np.abs(spec[:N//2])**2
plt.plot(power_spec[0:3000])

#Zero Crossing Rate 
#zero_crosses_rate = np.nonzero(np.diff(samples > 0))[0]
#print(zero_crosses_rate)

index = np.where(power_spec == np.max(power_spec))
print('max value index: ', index[0])

In [ ]:
abs_ceps = np.abs(ceps[:N//2])**2
#plt.plot(abs_ceps[:10])
np.max(abs_ceps)

In [ ]:
print("Power Spectrum: ", index[0], "\n")
print("Power Cepstrum: ", round(np.max(abs_ceps)))